In [5]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "security-alerts") \
  .load()

df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

DataFrame[key: string, value: string]

In [6]:
d1 = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").writeStream.outputMode("append").format("console").start()

25/04/17 07:05:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-e598351f-0969-4e85-94c4-a899510c7296. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/17 07:05:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/17 07:05:56 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+
|key|value|
+---+-----+
+---+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+----+--------------------+
| key|               value|
+----+--------------------+
|null|yeeahhhhhhhhhhhhh...|
+----+--------------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+----+--------------------+
| key|               value|
+----+--------------------+
|null|yeeahhhhhhhhhhhhh...|
+----+--------------------+



In [10]:
query.stop()

In [11]:
from pyspark.sql.functions import *
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "security-alerts") \
  .load()
kv_df=df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

words=kv_df.select(
   explode(
       split(kv_df.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

query =wordCounts.writeStream.outputMode("complete").format("console").start()


25/04/17 07:14:47 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0de0c79d-abe1-48a2-af35-0a803ea8e014. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/17 07:14:47 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/17 07:14:47 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+

-------------------------------------------
Batch: 3
-------------------------------------------
+----+------------------+
| key|             value|
+----+------------------+
|null|My name is Radhika|
+----+------------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+----+------------------+
| key|             value|
+----+------------------+
|null|My name is Radhika|
+----+------------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|   name|    1|
|     is|    1|
|Radhika|    1|
|     My|    1|
+-------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
-------------------------------------------
Batch: 4
-------------------------------------------
+----+--------------------+
| key|               value|
+----+--------------------+
|null|helloo hello Hi H...|
+----+--------------------+

+----+--------------------+
| key|               value|
+----+--------------------+
|null|helloo hello Hi H...|
+----+--------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+---------+-----+
|     word|count|
+---------+-----+
|     name|    1|
|   yeahhh|    1|
|    hello|    1|
|       Hi|    4|
|       is|    1|
|  Radhika|    1|
|   helloo|    1|
|yeahhhhhh|    1|
|       My|    1|
+---------+-----+

